<font size="10" color="black">Amostragem</font>

Eduardo Chaves Ferreira

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import pandas as pd
import scipy.stats as stat
from collections import Counter
from scipy.stats import t
from scipy.stats import norm

import os

path = os.environ['PATH']

if path.startswith('C'):
    IN_KAGGLE = False
else:
    IN_KAGGLE = True

In [ ]:
random.seed(1)
np.random.seed(1)
random_state = 1

In [ ]:
def calcula_Z_tstudent(confianca, tamanho_amostra):
    mean = 0
    std = 1
    rv = t(df=(tamanho_amostra-1))
    return rv.interval(confianca)[1]

In [ ]:
def calcula_Z_normal(confianca):
    mean = 0
    std = 1
    rv = norm(loc=mean, scale=std)
    return rv.interval(confianca)[1]

In [ ]:
def calcula_Z_Multiplas_Proporcoes(confianca, numero_proporcoes):
    area = math.pow(  confianca, 1/(numero_proporcoes-1) )
    return calcula_Z_normal(area)

In [ ]:
def recupera_amostra(populacao, tamanho_amostra):
    tamanho_amostra = int(tamanho_amostra)
    if type(populacao) is pd.DataFrame:
        return populacao.sample(tamanho_amostra)
    else:
        return [populacao[i] for i in np.random.randint(0, len(populacao), tamanho_amostra)]

In [ ]:
def calcula_intervalo_media(amostra, confianca, tamanho_populacao):
    tamanho_amostra = len(amostra)
    media_amostra = np.mean(amostra)
    desvio_amostra = np.std(amostra)
    numero_desvios = calcula_Z_tstudent(confianca, tamanho_amostra)
    desvio_amostras = desvio_amostra/np.sqrt(tamanho_amostra)
    margem_erro = numero_desvios*desvio_amostras  
    if tamanho_amostra>0.05*tamanho_populacao:
        margem_erro = margem_erro * (np.sqrt(tamanho_populacao-tamanho_amostra)/np.sqrt(tamanho_populacao-1))
    inferior = media_amostra-margem_erro
    superior = media_amostra+margem_erro
    return inferior,superior

In [ ]:
def calcula_intervalo_proporcao(amostra, confianca, valor, tamanho_populacao):
    tamanho_amostra = len(amostra)
    proporcao_valor = Counter(amostra)[valor]/tamanho_amostra
    numero_desvios = calcula_Z_tstudent(confianca, tamanho_amostra)
    margem_erro = numero_desvios*np.sqrt(proporcao_valor*(1-proporcao_valor))/np.sqrt(tamanho_amostra)
    if tamanho_amostra>0.05*tamanho_populacao:
        margem_erro = margem_erro * (np.sqrt(tamanho_populacao-tamanho_amostra)/np.sqrt(tamanho_populacao-1))
    inferior = proporcao_valor-margem_erro
    superior = proporcao_valor+margem_erro
    return inferior,superior

In [ ]:
def Tamanho_Amostra_Valor_Pontual(Tamanho_Populacao, Numero_Desvios, Variancia, Margem_Erro):
    return math.ceil( 
            (Tamanho_Populacao * math.pow(Numero_Desvios,2) * Variancia) / \
            ( (Tamanho_Populacao-1)*math.pow(Margem_Erro,2) + math.pow(Numero_Desvios,2)*Variancia )
            )

In [ ]:
def Tamanho_Amostra_Proporcao_Simples(Tamanho_Populacao, Numero_Desvios, Proporcao, Margem_Erro):
    return math.ceil( 
            (Tamanho_Populacao * math.pow(Numero_Desvios,2) * Proporcao * (1-Proporcao)) / \
            ( (Tamanho_Populacao-1)*math.pow(Margem_Erro,2) + math.pow(Numero_Desvios,2)*Proporcao*(1-Proporcao) )
            )

In [ ]:
def Tamanho_Amostra_Proporcao_Multipla(Tamanho_Populacao, Confianca, Numero_Proporcoes, Margem_Erro):
    Confianca_ = calcula_Z_Multiplas_Proporcoes(Confianca, Numero_Proporcoes)
    return math.ceil( 
            (Tamanho_Populacao * math.pow(Confianca_,2) * 0.25 ) / \
            ( (Tamanho_Populacao-1)*math.pow(Margem_Erro,2) + math.pow(Confianca_,2)*0.25 )
            )

In [ ]:
eleitoresRegiao = [21389000,63887000,11555000,39230000,10595000]
populacao = pd.DataFrame( columns=['Regiao','Voto'])
populacao.Voto = \
['B']*int(0.57*eleitoresRegiao[0]) + ['H']*int(0.20*eleitoresRegiao[0]) + ['C']*int(0.09*eleitoresRegiao[0]) +\
['B']*int(0.53*eleitoresRegiao[1]) + ['H']*int(0.19*eleitoresRegiao[1]) + ['C']*int(0.12*eleitoresRegiao[1]) +\
['B']*int(0.43*eleitoresRegiao[2]) + ['H']*int(0.37*eleitoresRegiao[2]) + ['C']*int(0.09*eleitoresRegiao[2]) +\
['B']*int(0.26*eleitoresRegiao[3]) + ['H']*int(0.51*eleitoresRegiao[3]) + ['C']*int(0.17*eleitoresRegiao[3]) +\
['B']*int(0.58*eleitoresRegiao[4]) + ['H']*int(0.21*eleitoresRegiao[4]) + ['C']*int(0.10*eleitoresRegiao[4]) 
populacao.Regiao = ['Sul']*(int(0.57*eleitoresRegiao[0]) + int(0.20*eleitoresRegiao[0]) + int(0.09*eleitoresRegiao[0])) + \
['Sudeste']*(int(0.53*eleitoresRegiao[1]) + int(0.19*eleitoresRegiao[1]) + int(0.12*eleitoresRegiao[1])) + \
['Norte']*(int(0.43*eleitoresRegiao[2]) + int(0.37*eleitoresRegiao[2]) + int(0.09*eleitoresRegiao[2])) + \
['Nordeste']*(int(0.26*eleitoresRegiao[3]) + int(0.51*eleitoresRegiao[3]) + int(0.17*eleitoresRegiao[3])) + \
['CentroOeste']*(int(0.58*eleitoresRegiao[4]) + int(0.21*eleitoresRegiao[4]) + int(0.10*eleitoresRegiao[4])) 

In [ ]:
Tamanho_Populacao = len(populacao)

In [ ]:
from sklearn.utils import shuffle
populacao = shuffle(populacao)
populacao.head()

In [ ]:
populacao['Regiao'].value_counts().plot(kind='bar')
plt.xlabel('Regiao')
plt.ylabel('Quantidade')

plt.title('Regiao')
plt.grid(True)
plt.show()

In [ ]:
populacao['Regiao'].value_counts()/Tamanho_Populacao

## Defina o tamanho da amostra

In [ ]:
Grau_Confianca = 0.95
Margem_Erro = 0.02
Tamanho_Amostra_Eleicao=Tamanho_Amostra_Proporcao_Multipla(
    Tamanho_Populacao,
    Grau_Confianca,
    3,
    Margem_Erro
)
Tamanho_Amostra_Eleicao

## Recupere uma amostra com o tamanho acima

In [ ]:
amostra = recupera_amostra(populacao, Tamanho_Amostra_Eleicao)
amostra = amostra.Voto

## Calcule o intervalo de confiânça para o candidato 'B'

In [ ]:
inferior,superior=calcula_intervalo_proporcao(amostra, Grau_Confianca, 'B', Tamanho_Populacao)
print('Intervalo de confiança B {} {}'.format(inferior,superior))
print('Margem de erro {}'.format((superior-inferior)/2))

## Calcule o intervalo de confiânça para o candidato 'H'

In [ ]:
inferior,superior=calcula_intervalo_proporcao(amostra, Grau_Confianca, 'H', Tamanho_Populacao)
print('Intervalo de confiança H {} {}'.format(inferior,superior))
print('Margem de erro {}'.format((superior-inferior)/2))

## Calcule o intervalo de confiânça para o candidato 'C'

In [ ]:
inferior,superior=calcula_intervalo_proporcao(amostra, Grau_Confianca, 'C', Tamanho_Populacao)
print('Intervalo de confiança C {} {}'.format(inferior,superior))
print('Margem de erro {}'.format((superior-inferior)/2))